# Практическое задание 1

# Ранжирование вопросов StackOverflow с помощью векторных представлений слов

## курс "Математические методы анализа текстов"


### ФИО: <впишите>

## Введение

В этом задании вы научитесь вычислять близость текстов и применить этот метод для поиска похожих вопросов на [StackOverflow](https://stackoverflow.com).

### Используемые библиотеки

В данном задании потребуются следующие библиотеки:
- [Gensim](https://radimrehurek.com/gensim/) — инструмент для решения различных задач NLP (тематическое моделирование, представление текстов, ...).
- [Numpy](http://www.numpy.org) — библиотека для научных вычислений.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — библилиотека с многими реализованными алгоритмами машинного обучения для анализа данных.
- [Nltk](http://www.nltk.org) — инструмент для работы с естественными языками.

Для выполнения бонусной части потребуется:
- [StarSpace](https://github.com/facebookresearch/StarSpace) — универсальная модель для обучения различных векторных представлений, разработанная командой Facebook.


### Данные

Данные лежат в архиве `StackOverflowData.zip`, который состоит из:
- `train.tsv` - обучающая выборка. В каждой строке через табуляцию записаны дублирующие друг друга предложения;
- `test.tsv` - тестовая выборка. В каждой строке через табуляцию записаны: *<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...*

Скачать архив можно здесь: [ссылка на google диск](https://drive.google.com/open?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_)

### Вектора слов/

Для решения вам потребуются две модели векторных представлений слов:

 - [Предобученные векторные представления слов](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit), которые были обучены с помощью стандартной модели word2vec на данных Google News (100 миллиардов слов). Модель содержит 300-мерные вектора для 3 миллионов слов и фраз. Вы можете скачать их, запустив блок кода ниже.
 - Векторные представления слов, полученные с помощью StarSpace на выборке StackOverflow. Вам потребуется обучить эту модель самим во второй части задания.

In [124]:
from functools import reduce
import gensim
import numpy as np
import os
import re
import torch
import torch.nn as nn

In [ ]:
# Download Google vectors to directory *target_dir*

from download_utils import download_google_vectors
download_google_vectors(target_dir='.')

## Часть 1. Предобученные векторные представления слов (2 балла)

Скачайте предобученные вектора и загрузите их с помощью функции [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) библиотеки Gensim с параметром *binary=True*. Если суммарный размер векторов больше, чем доступная память, то вы можете загрузите только часть векторов, указав параметр *limit* (рекомендуемое значение: 500000).

In [2]:
os.listdir()

['download_utils.py',
 'stackoverflow_similar_questions.zip',
 'lab_word_embeddings.ipynb',
 'tests.py',
 'GoogleNews-vectors-negative300.bin',
 'GoogleNews-vectors-negative300.bin.gz',
 'data',
 '__pycache__',
 '.ipynb_checkpoints']

In [3]:
# TODO

wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin',
    binary=True,
    limit=500000
)

In [8]:
'dog' in wv_embeddings

True

In [9]:
type(wv_embeddings['dog'])

numpy.ndarray

In [10]:
EMBEDDING_VECTOR_SIZE = wv_embeddings['dog'].size

In [11]:
EMBEDDING_VECTOR_SIZE

300

### Как пользоваться этими векторами?

Как только вы загрузите векторные представления слов в память, убедитесь, что имеете к ним доступ. Сначала вы можете проверить, содержится ли какое-то слово в загруженных эмбедингах:

    'word' in wv_embeddings

Затем, чтобы получить соответствующий вектор, вы можете использовать оператор доступа по ключу:

    wv_embeddings['word']

### Проверим, корректны ли векторные представления

Чтобы предотвратить возможные ошибки во время первого этапа, можно проверить, что загруженные вектора корректны. Для этого вы можете запустить функцию *check_embeddings*. Она запускает 3 теста:
1. Находит наиболее похожие слова для заданных "положительных" и "отрицательных" слов.
2. Находит, какое слово из заданного списка не встречается с остальными.
3. Находит наиболее похожее слово для заданного.

In [7]:
from tests import check_embeddings

print(check_embeddings(wv_embeddings))

These embeddings look good.


### Векторные представления текста

Чтобы перейти от отдельных слов к векторным представлениям вопросов, предлагается подсчитать **среднее** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектоора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор. 

In [ ]:
import numpy as np

In [191]:
# TODO

def question_to_vec_by_mean(
    question, embeddings, dim=EMBEDDING_VECTOR_SIZE):
    """
        question: a string
        embeddings: dict where the key is a word and a value is its' embedding
        dim: size of the representation

        result: vector representation for the question
    """
    question = re.sub('\s+', ' ', question)
    words = question.split(' ')
    zero_vector = np.zeros((dim,))
    sum_embedding = np.zeros((dim,))
    num_words_with_known_embegginds = 0
    
    for word in words:
        if word in embeddings:
            word_embedding = embeddings.get_vector(word)
            num_words_with_known_embegginds += 1
        else:
            word_embedding = zero_vector  # no need to copy np.array
        
        sum_embedding += word_embedding
    
    return sum_embedding / max(1, num_words_with_known_embegginds)

Для базовой проверки решения запустите клетку ниже.

In [192]:
from tests import question_to_vec_tests

print(question_to_vec_tests(question_to_vec_by_mean, wv_embeddings))

Basic tests are passed.


Теперь у нас есть метод для создания векторного представления любого предложения. Оценим, как будет работать это решение.

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из *N* вопросов *R* случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели *R + 1* примеров и смотреть на позицию дубликата.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)],$$
где $q_i$ - $i$-ый вопрос, $dup_i$ - его дубликат, $topK(q_i)$ - первые *K* элементов в ранжированном списке, который выдает наша модель.

#### DCG@K
Второй метрикой будет упрощенная [DCG метрика](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K],$$
где $rank_{dup_i}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$. С такой метрикой модель штрафуется за низкую позицию корректного ответа.

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. Пусть $N = 1$, $R = 3$, вопрос $q_1$ это "Что такое python", а его дубликат $dup_1$ это "Что такое язык python". Пусть модель выдала следующий ранжированный список кандидатов:

1. *"Как узнать с++"*
2. *"Что такое язык python"*
3. *"Хочу учить Java"*
4. *"Не понимаю Tensorflow"*

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [dup_1 \in top1(q_1)] = 0$
- [K = 4] $\text{Hits@4} =  [dup_1 \in top4(q_1)] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

<font color='red'>**Реализуйте функции *hits_count* и *dcg_score*. **</font> 

Каждая функция имеет два аргумента: *dup_ranks* и *k*. *dup_ranks* является списком, который содржит *рейтинги дубликатов* (их позиции в ранжированном списке). Например, *dup_ranks = [2]* для примера, описанного выше.

In [193]:
# TODO

def hits_count(dup_ranks, k):
    """
        dup_ranks: list of ranks of the duplicates; one rank per question; 
                   length is a number of questions that we check (N); 
                   rank is a number from 1 to len(candidates for the question).
        k: number of top-ranked elements (k in Hits@k metric)

        result: return Hits@k value for the current ranking.
    """
    num_questions = len(dup_ranks)
    num_k_hits = sum(rank <= k for rank in dup_ranks)
    
    return num_k_hits / max(1, num_questions)

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K],$$

In [194]:
# TODO

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list of ranks of the duplicates; one rank per question; 
                   length is a number of questions that we check (N); 
                   rank is a number from 1 to len(candidates for the question).
        k: number of top-ranked elements (k in DCG@k metric)

        result: return DCG@k value for the current ranking.
    """
    num_questions = len(dup_ranks)
    num_k_dcg = sum(
        1.0 / np.log2(1 + rank)
        for rank in dup_ranks
        if rank <= k
    )
    
    return num_k_dcg / num_questions

Протестируйте функции. Успешное прохождение базовых тестов еще не гарантирует корректности реализации!

In [195]:
from tests import test_hits

print(test_hits(hits_count))

Basic test are passed.


In [196]:
from tests import test_dcg

print(test_dcg(dcg_score))

Basic test are passed.


### Ранжирование вопросов StackOverflow

Выборка уже разбита на обучающую и тестовую. Все файлы используют табуляцию в качестве разделителя, но они имеют разный формат:

- *обучающая* выборка (test.tsv) содержит похожие друг на друга предложения в каждой строке;
- *тестовая* выборка (validation.tsv) содержит в каждой строке: *вопрос, похожий вопрос, отрицательный пример 1, отрицательный пример 2, ...*

Считайте тестовую выборку для оценки качества текущего решения.

In [197]:
def read_corpus(filename):
    data = []
    
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
        
    return data

In [198]:
data_path = 'data'

In [199]:
os.listdir(data_path)

['train.tsv', '.DS_Store', 'validation.tsv']

In [200]:
# TODO

validation = read_corpus(
    os.path.join(data_path, 'validation.tsv')
)

KeyboardInterrupt: 

In [38]:
validation[0][:5]

['How to print a binary heap tree without recursion?',
 'How do you best convert a recursive function to an iterative one?',
 'How can i use ng-model with directive in angular js',
 'flash: drawing and erasing',
 'toggle react component using hide show classname']

<font color='red'>**Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния.**</font>
    
Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список *[(2, c), (0, a), (1, b)]*.

In [41]:
from sklearn.metrics.pairwise import cosine_similarity

In [201]:
# TODO

def rank_candidates(
    question, candidates, embeddings, dim=EMBEDDING_VECTOR_SIZE,
    question_to_vec=question_to_vec_by_mean):
    """
        question: a string
        candidates: a list of strings (candidates) which we want to rank
        embeddings: some embeddings
        dim: dimension of the current embeddings
        
        result: a list of pairs (initial position in the list, question)
    """
    question_embedding = question_to_vec(question, embeddings)
    similarities = []
    
    for candidate in candidates:
        candidate_embedding = question_to_vec(candidate, embeddings)
        
        similarities.append(
            cosine_similarity(
                question_embedding.reshape(1, -1),
                candidate_embedding.reshape(1, -1)
            )[0][0]
        )
    
    original_positions = list(range(len(candidates)))
    sort_indices = np.argsort(similarities).tolist()[::-1]

    return list(zip(
        np.array(original_positions)[sort_indices].tolist(),
        np.array(candidates)[sort_indices].tolist()
    ))

Протестируйте работу функции на примерах ниже.

In [53]:
from tests import test_rank_candidates

print(test_rank_candidates(rank_candidates, wv_embeddings))

Basic tests are passed.


Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут).

In [98]:
def compute_wv_ranking(validation, rank_candidates):
    wv_ranking = []

    for line in validation:
        q, *ex = line
        ranks = rank_candidates(q, ex)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    
    return wv_ranking

In [58]:
len(validation)

3760

In [56]:
%%time

wv_ranking = []

for line in validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

KeyboardInterrupt: 

In [61]:
def print_k_metrics(wv_ranking, k_list=None):
    if k_list is None:
        k_list = [1, 5, 10, 100, 500, 1000]

    for k in k_list:
        print(
            "DCG@%4d: %.3f | Hits@%4d: %.3f" %
            (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k))
        )

In [65]:
print_k_metrics(wv_ranking)

DCG@   1: 0.212 | Hits@   1: 0.212
DCG@   5: 0.264 | Hits@   5: 0.310
DCG@  10: 0.281 | Hits@  10: 0.363
DCG@ 100: 0.320 | Hits@ 100: 0.559
DCG@ 500: 0.353 | Hits@ 500: 0.816
DCG@1000: 0.372 | Hits@1000: 1.000


Если вы проделали все шаги правильно, то вы должны разочароваться полученными результатами. Давайте попробуем понять, почему качество модели такое низкое. Когда вы работаете с какими-либо данными, очень полезно первым делом посмотреть на них глазами. Выведите несколько вопросов из наших данных:

In [66]:
for line in validation[:3]:
    q, *examples = line
    
    print(q, *examples[:3])
    print()

How to print a binary heap tree without recursion? How do you best convert a recursive function to an iterative one? How can i use ng-model with directive in angular js flash: drawing and erasing

How to start PhoneStateListener programmatically? PhoneStateListener and service Java cast object[] to model WCF and What does this mean?

jQuery: Show a div2 when mousenter over div1 is over when hover on div1 depenting on if it is on div2 or not it should act differently How to run selenium in google app engine/cloud? Python Comparing two lists of strings for similarities



## Часть 2. Предобработка данных (1 балл)

Как вы могли заметить, мы имеем дело с сырыми данными. Это означает, что там присутствует много опечаток, спецсимволов и заглавных букв. В нашем случае это все может привести к ситуации, когда для данных токенов нет предобученных векторов. Поэтому необходима предобработка.

<font color='red'>**Реализуйте функцию предобработки текстов.**</font>

Вам требуется:
- Перевести символы в нижний регистр;
- Заменить символы пунктуации на пробелы;
- Удалить "плохие" символы;
- Удалить стопслова.

In [67]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/alvant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [78]:
re.sub(r'[\[\]]', '!', r'[a\a]')

'!a\\a!'

In [118]:
# TODO

def text_prepare(text, stopwords=stopwords.words()):
    """
        text: a string
        
        return: modified string
    """
    text = text.lower()
    text = re.sub(r'[,;:?!."\'\-+*<>{}\[\]()/\\#@]`', ' ', text) # TODO
    text = re.sub('\s+', ' ', text)
    
    # TODO: bad symbols
    
    words = text.split(' ')
    words = [w for w in words if w not in stopwords]
    
    return ' '.join(words)

<font color='red'>**Теперь преобразуйте все вопросы из тестовой выборки. Оцените, как изменилось качество. Сделайте выводы.**</font>

In [119]:
text = 'Hello! ha-ha, so cute... Where do we go? On the table?! #@'

print('Before:', text)
print('After :', text_prepare(text))

Before: Hello! ha-ha, so cute... Where do we go? On the table?! #@
After : hello cute go table 


In [95]:
# TODO

processed_validation = []

for sentences in validation:
    processed_sentences = []
    
    for sentence in sentences:
        processed_sentences.append(
            text_prepare(sentence)
        )
    
    processed_validation.append(processed_sentences)

KeyboardInterrupt: 

In [96]:
len(processed_validation)

270

In [99]:
# TODO

wv_ranking = compute_wv_ranking(
    processed_validation,
    rank_candidates=lambda q, cands: rank_candidates(
        q, cands, wv_embeddings)
)

In [100]:
# TODO

print_k_metrics(wv_ranking)

DCG@   1: 0.322 | Hits@   1: 0.322
DCG@   5: 0.388 | Hits@   5: 0.448
DCG@  10: 0.402 | Hits@  10: 0.493
DCG@ 100: 0.441 | Hits@ 100: 0.681
DCG@ 500: 0.462 | Hits@ 500: 0.848
DCG@1000: 0.478 | Hits@1000: 1.000


## Часть 3. Представления для неизвестных слов. (2 балла)

<font color='red'>**Оцените долю слов в выборке, для которых нет эмбеддинга в модели.**</font>

In [126]:
# TODO

def count_number_of_words_without_embedding(sentences, wv_embeddings):
    words = set()
    words_without_embedding = set()
    
    for sentence in sentences:
        for word in sentence.split():
            words.add(word)
            
            if word not in wv_embeddings:
                words_without_embedding.add(word)
    
    return len(words_without_embedding), len(words), words, words_without_embedding

In [129]:
limit = 100

num_without, num_words, all_words, words_without = count_number_of_words_without_embedding(
    reduce(lambda reduced, cur: reduced + cur, validation[:limit], []),
    wv_embeddings
)

print(f'Validation:\nNum words {num_words}, num without {num_without} (fraction {num_without / num_words:.3f})')

num_without, num_words, all_words, words_without = count_number_of_words_without_embedding(
    reduce(lambda reduced, cur: reduced + cur, processed_validation[:limit], []),
    wv_embeddings
)

print(f'After processing:\nNum words {num_words}, num without {num_without} (fraction {num_without / num_words:.3f})')

Validation:
Num words 37078, num without 25433 (fraction 0.686)
After processing:
Num words 18010, num without 10324 (fraction 0.573)


In [130]:
known_words = all_words.difference(words_without)

Для того, что получить представления для неизвестного слова, воспользуемся следующим подходом:
    
1. Будем восстанавливать эмбеддинг неизвестного слова как сумму эмбеддингов буквенных триграмм. Например, слово where должно представляться суммой триграмм _wh, whe, her, ere, re_

2. В качестве обучающих данных будем использовать слова, для которых есть эмбеддинг в модели. Будем обучать эмбеддинги триграмм по выборке эмбеддингов с помощью функционала MSE:

$$L = \sum_{w \in W_{known}}\| f_{\theta}(w) - v_w \|^2 \to \min_{\theta}$$

где:

* $W_{known}$ — множество известных модели слов
* $f_{\theta}(w)$ — сумма эмбеддингов триграмм слова $w$
* $v_w$ — эмбеддинг слова $w$
* $\theta$ — веса эмбеддингов триграмм

<font color='red'>**Реализуйте предложенную модель ниже.**</font>

Используйте класс nn.EmbeddingBag для построения среднего вектора представлений.

In [147]:
# TODO

class TrigrammEmbeddingsModel(nn.Module):
    def __init__(self, all_known_tokens, embedding_dim=EMBEDDING_VECTOR_SIZE):
        """
        all_known_tokens : list of str
        embedding_dim : int
        """
        super().__init__()
        
        self._all_known_tokens = list(all_known_tokens)
        self._embedding_dim = embedding_dim
        
        self.embedding = nn.EmbeddingBag(
            len(self._all_known_tokens),
            self._embedding_dim,
            mode='sum'
        )
        
        self._ngramm_length = 3
    
    def forward(self, token):
        trigramms = []
        
        # where
        # wh, whe, her, ere, re
        # -1 .... ........ ..., len(where)-2
        for i in range(-1, len(token) - 1):
            trigramms.append(token[
                max(0, i) : i+self._ngramm_length
            ])
        
        trigramm_indices = [
            self._all_known_tokens.index(t)
            for t in trigramms
            if t in self._all_known_tokens
        ]
        
        return self.embedding(
            torch.LongTensor(trigramm_indices),
            torch.LongTensor([0])
        )

<font color='red'>** Обучите модель. Оцените, как изменилось качество. Сделайте выводы.**</font>

Если вы всё реализовали правильно, качество решения должно вырасти.

In [153]:
len(known_words)

7686

In [155]:
trigram_embeddings_sum = trigramm_model('cat')
target_embedding = torch.FloatTensor(wv_embeddings['cat'])

In [154]:
# TODO

trigramm_model = TrigrammEmbeddingsModel(known_words)
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(trigramm_model.parameters(), lr=1e-4)

for word_index, word in enumerate(known_words):
    trigram_embeddings_sum = trigramm_model(word)
    target_embedding = torch.FloatTensor(wv_embeddings[word])
    
    loss = criterion(trigram_embeddings_sum, target_embedding)
    
    if word_index % 500 == 0:
        print(word, loss.item())

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

/home/alvant/anaconda3/lib/python3.6/site-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([300])) that is different to the input size (torch.Size([1, 300])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


expiring 1049.76171875
pages 589.0363159179688
chart 872.8565673828125
chin 277.09149169921875
busboy 627.1744995117188
spacing 1144.5872802734375
reassigning 503.1912536621094
hits 753.578369140625
declarative 316.7134704589844
left 324.7681884765625
aligned 8.96114730834961
thing 731.9093017578125
hammer 8.619854927062988
panorama 1431.4688720703125
injecting 380.9019470214844
instance 326.5602722167969


In [208]:
class WvEmbeddingsViaTrigramms:
    def __init__(self, trigramm_model: TrigrammEmbeddingsModel):
        self._model = trigramm_model
    
    def get_vector(self, word):
        return self._model(word).detach().numpy().flatten()
    
    def __contains__(self, word):
        return True

In [213]:
# TODO

limit = 100

wv_ranking = compute_wv_ranking(
    processed_validation[:limit],
    rank_candidates=lambda q, cands: rank_candidates(
        q, cands, WvEmbeddingsViaTrigramms(trigramm_model))
)

In [210]:
len(wv_ranking)

270

In [100]:
# TODO

print_k_metrics(wv_ranking)

DCG@   1: 0.322 | Hits@   1: 0.322
DCG@   5: 0.388 | Hits@   5: 0.448
DCG@  10: 0.402 | Hits@  10: 0.493
DCG@ 100: 0.441 | Hits@ 100: 0.681
DCG@ 500: 0.462 | Hits@ 500: 0.848
DCG@1000: 0.478 | Hits@1000: 1.000


In [211]:
# TODO

print_k_metrics(wv_ranking)

DCG@   1: 0.322 | Hits@   1: 0.322
DCG@   5: 0.388 | Hits@   5: 0.448
DCG@  10: 0.402 | Hits@  10: 0.493
DCG@ 100: 0.441 | Hits@ 100: 0.681
DCG@ 500: 0.462 | Hits@ 500: 0.848
DCG@1000: 0.478 | Hits@1000: 1.000


## Бонусная часть: векторные представления StarSpace (2 балла)

В бонусной части вам предлгается обучить эмбеддинги специально для задачи поиска дубликатов с помощью пакета [StarSpace](https://github.com/facebookresearch/StarSpace). К сожалению, его нельзя запустить на Windows, поэтому в этом случае мы рекоммендуем использовать готовый [docker container](https://github.com/hse-aml/natural-language-processing/blob/master/Docker-tutorial.md) с пошаговыми инструкциями или воспользоваться платформой google colab.

Данная модель все еще представляет вопросы с помощью усреднения векторов слов, однако обучается по размеченной выборке пар близких вопросов. Это позволяет обучить вектора, которые лучше подходят для конкретной задачи. Напомним, что в модели word2vec обучение происходят по парам близких слов, и на этапе обучения модель ничего не знает о наших планах по их усреднению в пост-обработке.


### Как выбрать  параметры модели?

Ниже приведены некоторые рекомендации, с которых можно начать свои эксперименты.

- Обучение на парах близких предложений соответствует режиму *trainMode = 3*.
- Используйте метод оптимизации adagrad (параметр *adagrad=True*).
- Установите длину фразы равной 1 (параметр *ngrams*), чтобы получить только вектора слов.
- Не используйте большое количество эпох (5 должно быть достаточно).
- Поэкспериментируйте с несколькими размерностями *dim* (например, от 100 до 300).
- Для сравнения векторов используйте *косинусную меру*.
- Установите *minCount* больше 1 (например, 2), если вы не хотите получить вектора для редко встречающихся слов.
- Параметр *verbose=True* будет показывать вам прогресс процесса обучения.
- Параметр *negSearchLimit* отвечает за число отрицательных примеров, которые используются в обучении, рекомендованное значение 10.
- Для ускорения обучения мы рекоммендуем поставить *шаг обучения (learning rate)* равным 0.05.

<font color='red'>** Обучите вектора StarSpace для униграм на обучающей выборке. Не забудьте использовать предобработанную версию данных. **</font>

Если вы следовали инструкциям правильно, то процесс обучения займет около 1 часа. Размер словаря полученных векторных представлений должен быть порядка 100000 (число строк в полученном файле). 

In [ ]:
from starspace import 

In [ ]:
##### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ TRAINING HAPPENING HERE #####

Ниже вы можете проверить качество работы вашей модели. Так как обучение происходило для конкретной задачи на размеченных данных, то ожидается, что это решение будет иметь более высокое качество.

In [ ]:
starspace_embeddings = ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

In [ ]:
ss_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ss_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking, k), 
                                              k, hits_count(ss_prepared_ranking, k)))

<font color='red'>**Опишите результаты ваших экспериментов, сделайте выводы.**</font>

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################